In [ ]:
import json
import random
import ollama
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import openai
import numpy as np
import random
import llama_models
from scipy.stats import spearmanr
import ollama

# Set your OpenAI API key

# Load JSONL data
def load_jsonl(file_path):
    with open(file_path, 'r') as file:
        data = [json.loads(line) for line in file]
    return data

# Reconstruct SQL query from dataset
def reconstruct_query(entry, tables_map):
    table = tables_map[entry['table_id']]
    headers = table['header']
    sql = entry['sql']
    
    select_column = headers[sql['sel']]
    aggregation = ["", "MAX", "MIN", "COUNT", "SUM", "AVG"][sql['agg']]
    conditions = sql['conds']
    
    query = f"SELECT {aggregation}({select_column})".strip()
    if conditions:
        condition_clauses = []
        for col_index, operator, value in conditions:
            col_name = headers[col_index]
            operators = ["=", ">", "<", "OP"][operator]
            condition_clauses.append(f"{col_name} {operators} {value}")
        query += " WHERE " + " AND ".join(condition_clauses)
    return query

# Generate prompt for the LLM
def generate_prompt(sql_query):
    prompt = f"""
    Translate the following SQL query into natural language: {sql_query}.
    Include only the most possible translation and do not innclude any reasoning.
    """
    return prompt

def sql2nl(dev_path, tables_path, model):
    # Load datasets
    dev_data = load_jsonl(dev_path)
    tables_data = load_jsonl(tables_path)

    # Map table_id to table metadata
    tables_map = {table['id']: table for table in tables_data}

    # Evaluate LLM on dev data
    results = []

    # Sample 10 examples for evaluation
    sampled_entries = random.sample(dev_data, 100)

    for entry in sampled_entries:
        sql_query = reconstruct_query(entry, tables_map)
        prompt = generate_prompt(sql_query)
        
        if model == "llama":
            response = ollama.chat(
                model="llama3.2",
                messages=[
                    {"role": "system", "content": "You are an expert in SQL and natural language translation."},
                    {"role": "user", "content": prompt}
                ],
            )
            gpt_reply = response["message"]["content"]
        elif model == "gpt-4o":
            response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an expert in statistics and time series analysis."},
                {"role": "user", "content": prompt}
            ]
            )
            
            # Extract and parse the suggested parameters
            gpt_reply = response['choices'][0]['message']['content']

        # Append results
        results.append({
            "question": entry["question"],
            "sql_query": sql_query,
            "gpt_reply": gpt_reply
        })

    # Display evaluation results
    for res in results:
        print(f"Original Question: {res['question']}")
        print(f"SQL Query: {res['sql_query']}")
        print(f"LLM Reply: {res['gpt_reply']}")
        print("-" * 50)

    # Compute BLEU scores
    def compute_bleu(reference, candidate):
        reference_tokens = nltk.word_tokenize(reference.lower())
        candidate_tokens = nltk.word_tokenize(candidate.lower())
        smoothing_function = SmoothingFunction().method1
        return sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=smoothing_function)

    # Calculate BLEU scores for each result
    bleu_scores = [
        compute_bleu(res["question"], res["gpt_reply"])
        for res in results
    ]

    # Display BLEU scores
    for i, score in enumerate(bleu_scores):
        print(f"Example {i+1} BLEU Score: {score:.2f}")

    # Calculate and display average BLEU score
    average_bleu = sum(bleu_scores) / len(bleu_scores)
    print(f"Average BLEU Score: {average_bleu:.2f}")
    return average_bleu, bleu_scores

In [ ]:

# Define paths to the dataset files
dev_path = "/home/qic69/Desktop/domain_metric/DMM_project/WikiSQL/data/dev.jsonl"
tables_path = "/home/qic69/Desktop/domain_metric/DMM_project/WikiSQL/data/dev.tables.jsonl"
model = "gpt-4o"



Original Question: What score to par did Mike Weir have?
SQL Query: SELECT (To par) WHERE Player = mike weir
LLM Reply: Retrieve the "To par" value for the player Mike Weir.
--------------------------------------------------
Original Question: what is the total number of films directy and written by john callaghan?
SQL Query: SELECT COUNT(Directed by) WHERE Written by = John Callaghan
LLM Reply: Count the number of entries where the writer is John Callaghan.
--------------------------------------------------
Original Question: what's the first epbeingode with final epbeingode being "rio"
SQL Query: SELECT (First Episode) WHERE Final Episode = "Rio"
LLM Reply: Retrieve the first episode for which the final episode is "Rio".
--------------------------------------------------
Original Question: What position does Zack Torquato play?
SQL Query: SELECT (Position) WHERE Player = zack torquato
LLM Reply: Retrieve the position of the player named Zack Torquato.
--------------------------------